**Setup and Installation**

In [ ]:
!pip install langchain langchain-openai llama-index haystack-ai tiktoken openai

import os
import json
import tiktoken
from typing import List, Dict, Any, Optional, Union

os.environ["OPENAI_API_KEY"] = "your-api-key"

from langchain_core.documents import Document

**Basic Utility Functions**

In [2]:
def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Count the number of tokens in a text string."""
    encoder = tiktoken.encoding_for_model(model)
    return len(encoder.encode(text))

def print_separator():
    """Print a visual separator."""
    print("\n" + "="*50 + "\n")

# Create some sample documents for testing
sample_docs = [
    Document(page_content="Paris is the capital of France. It is known for the Eiffel Tower and Louvre Museum.",
             metadata={"source": "geography_textbook", "page": 42}),
    Document(page_content="France is a country in Western Europe with a population of about 67 million people.",
             metadata={"source": "world_almanac", "page": 156}),
    Document(page_content="The Eiffel Tower was completed in 1889 and stands 330 meters tall.",
             metadata={"source": "landmarks_guide", "page": 28})
]

**Section 1: LangChain Prompt Templates**

In [ ]:
print("Section 1: LangChain Prompt Templates")

try:
    from langchain.prompts import PromptTemplate, ChatPromptTemplate
    from langchain.prompts.few_shot import FewShotPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import MessagesPlaceholder
    from langchain_openai import ChatOpenAI

    print("Successfully imported LangChain libraries")

    # Basic Prompt Template
    basic_template = PromptTemplate.from_template(
        """Answer the question based on the context.

        Context: {context}
        Question: {question}

        Answer:"""
    )

    print("\nBasic Template Example:")
    context = "\n".join([doc.page_content for doc in sample_docs])
    question = "What is the height of the Eiffel Tower?"

    formatted_prompt = basic_template.format(context=context, question=question)
    print(formatted_prompt)
    print(f"Tokens: {count_tokens(formatted_prompt)}")

    # Few-shot Template
    examples = [
        {"context": "Paris is the capital of France.",
         "question": "What is the capital of France?",
         "answer": "Paris is the capital of France."},
        {"context": "Berlin is the capital of Germany.",
         "question": "What is the capital of Germany?",
         "answer": "Berlin is the capital of Germany."}
    ]

    example_prompt = PromptTemplate.from_template(
        """Context: {context}
        Question: {question}
        Answer: {answer}"""
    )

    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="Answer the question based on the provided context:\n\n",
        suffix="\n\nContext: {context}\nQuestion: {question}\nAnswer:",
        input_variables=["context", "question"]
    )

    print("\nFew-shot Template Example:")
    formatted_few_shot = few_shot_prompt.format(context=context, question=question)
    print(formatted_few_shot)
    print(f"Tokens: {count_tokens(formatted_few_shot)}")

    # Chat Prompt Template
    chat_template = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that answers questions based on the provided context."),
        ("human", "Context: {context}\n\nQuestion: {question}")
    ])

    print("\nChat Template Example:")
    formatted_chat = chat_template.format_messages(context=context, question=question)
    print(formatted_chat)

    # RAG-specific Template
    rag_template = PromptTemplate.from_template(
        """You are an assistant for question-answering tasks.
        Use the following pieces of retrieved context to answer the question at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.
        Use three sentences maximum and keep the answer concise.

        CONTEXT:
        {context}

        QUESTION:
        {question}

        ANSWER:"""
    )

    print("\nRAG-specific Template Example:")
    formatted_rag = rag_template.format(context=context, question=question)
    print(formatted_rag)
    print(f"Tokens: {count_tokens(formatted_rag)}")

    # Creating a chain with the template
    print("\nBuilding a LangChain chain with the template:")

    if os.environ.get("OPENAI_API_KEY"):
        try:
            from langchain.chains import LLMChain
            from langchain_openai import OpenAI

            llm = OpenAI(temperature=0)
            chain = LLMChain(llm=llm, prompt=rag_template)

            print("Chain created successfully")
            print("Chain run would be: chain.run(context=context, question=question)")
        except Exception as e:
            print(f"Could not create LangChain chain: {e}")
    else:
        print("OpenAI API key not set, skipping chain creation")

except ImportError as e:
    print(f"Could not import LangChain: {e}")
    print("Install with: pip install langchain langchain-openai")

print_separator()

**Section 2: LlamaIndex Prompt Templates**

In [ ]:
print("Section 2: LlamaIndex Prompt Templates")

try:
    from llama_index.prompts import PromptTemplate as LlamaPromptTemplate

    print("Successfully imported LlamaIndex")

    # QA template
    qa_template = LlamaPromptTemplate(
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the query.\n"
        "Query: {query_str}\n"
        "Answer: "
    )

    # Refine template
    refine_template = LlamaPromptTemplate(
        "The original query is as follows: {query_str}\n"
        "We have provided an existing answer: {existing_answer}\n"
        "We have the opportunity to refine the existing answer "
        "with some more context below.\n"
        "------------\n"
        "{context_msg}\n"
        "------------\n"
        "Given the new context, refine the original answer to better "
        "answer the query. If the context isn't useful, return the original answer."
    )

    # Tree summarize template
    tree_summarize_template = LlamaPromptTemplate(
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information, generate a summary that captures the key points."
    )

    print("\nLlamaIndex QA Template:")
    context_str = "\n".join([doc.page_content for doc in sample_docs])
    query_str = "What is the height of the Eiffel Tower?"

    formatted_qa = qa_template.format(context_str=context_str, query_str=query_str)
    print(formatted_qa)
    print(f"Tokens: {count_tokens(formatted_qa)}")

    print("\nLlamaIndex Refine Template:")
    existing_answer = "The Eiffel Tower is in Paris, France."
    context_msg = "The Eiffel Tower was completed in 1889 and stands 330 meters tall."

    formatted_refine = refine_template.format(
        query_str=query_str,
        existing_answer=existing_answer,
        context_msg=context_msg
    )
    print(formatted_refine)
    print(f"Tokens: {count_tokens(formatted_refine)}")

    print("\nLlamaIndex allows easy customization of built-in templates:")
    custom_qa_template = LlamaPromptTemplate(
        "You are a knowledgeable assistant.\n"
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Answer the following query based ONLY on the context provided.\n"
        "If the context doesn't contain relevant information, say so.\n"
        "Query: {query_str}\n"
        "Answer: "
    )

    formatted_custom_qa = custom_qa_template.format(
        context_str=context_str,
        query_str=query_str
    )
    print(formatted_custom_qa)

except ImportError as e:
    print(f"Could not import LlamaIndex: {e}")
    print("Install with: pip install llama-index")

print_separator()

**Section 3: Haystack Prompt Templates**

In [ ]:
print("Section 3: Haystack Prompt Templates")

try:
    from haystack.nodes import PromptNode, PromptTemplate

    print("Successfully imported Haystack")

    # Basic Haystack template
    haystack_template = PromptTemplate(
        prompt="""
        Answer the question based on the given documents.
        Documents: {documents}
        Question: {query}
        Answer:
        """,
        output_parser=None
    )

    print("\nHaystack Template Example:")
    documents = "\n".join([doc.page_content for doc in sample_docs])
    query = "What is the height of the Eiffel Tower?"

    formatted_haystack = haystack_template.prepare(documents=documents, query=query)
    print(formatted_haystack)
    print(f"Tokens: {count_tokens(formatted_haystack)}")

    # Using PromptNode
    if os.environ.get("OPENAI_API_KEY"):
        try:
            prompt_node = PromptNode(
                model_name_or_path="gpt-3.5-turbo",
                default_prompt_template=haystack_template,
                api_key=os.environ.get("OPENAI_API_KEY")
            )
            print("\nHaystack PromptNode created successfully")
        except Exception as e:
            print(f"Could not create Haystack PromptNode: {e}")
    else:
        print("\nOpenAI API key not set, skipping PromptNode creation")

except ImportError as e:
    print(f"Could not import Haystack: {e}")
    print("Install with: pip install haystack-ai")

print_separator()

**Section 4: Custom Template Library for Domain-Specific Applications**

In [ ]:
print("Section 4: Custom Template Library for Domain-Specific Applications")

class MedicalRAGTemplates:
    """Custom template library for medical RAG applications."""

    @staticmethod
    def diagnosis_template(context, symptoms):
        return f"""
        You are a medical assistant providing information based on medical literature.
        Analyze the following symptoms using only the provided medical context.

        MEDICAL CONTEXT:
        {context}

        REPORTED SYMPTOMS:
        {symptoms}

        Based strictly on the medical context, provide:
        1. Possible conditions consistent with these symptoms
        2. Important missing information that would help narrow the possibilities
        3. Appropriate next steps based on medical guidelines

        Include citations to specific documents using [Doc X] notation.
        Emphasize that this is informational only and not a diagnosis.
        """

    @staticmethod
    def medication_info_template(context, medication):
        return f"""
        Provide information about the following medication using only the provided context.

        CONTEXT:
        {context}

        MEDICATION:
        {medication}

        Include information on:
        - Approved uses
        - Common side effects
        - Typical dosing
        - Major contraindications

        Cite sources as [Doc X] and include appropriate medical disclaimers.
        """

    @staticmethod
    def treatment_comparison_template(context, condition, treatments):
        return f"""
        Compare the following treatments for {condition} based only on the provided context.

        MEDICAL CONTEXT:
        {context}

        TREATMENTS TO COMPARE:
        {treatments}

        For each treatment, discuss:
        1. Efficacy based on clinical evidence
        2. Common side effects
        3. Contraindications
        4. Typical treatment protocol

        Format as a comparison table followed by a brief summary.
        Include citations and medical disclaimers.
        """

# Create medical document examples
medical_docs = [
    Document(page_content="Aspirin is commonly used for pain relief and prevention of heart attacks. Common side effects include stomach irritation and increased risk of bleeding. Typical dosing is 81mg to 325mg daily for cardiac prevention.",
             metadata={"source": "drug_reference", "page": 112}),
    Document(page_content="Symptoms of the common cold include runny nose, cough, sore throat, and mild fever. Treatment is typically supportive, including rest, hydration, and over-the-counter medications for symptom relief.",
             metadata={"source": "clinical_guidelines", "page": 43}),
    Document(page_content="Migraine treatment options include NSAIDs, triptans, and anti-nausea medications. Preventive treatments include beta-blockers, antidepressants, and anti-seizure medications.",
             metadata={"source": "neurology_textbook", "page": 215})
]

print("Medical RAG Templates Example:")

# Diagnosis template example
medical_context = "\n".join([doc.page_content for doc in medical_docs])
symptoms = "Severe headache, light sensitivity, nausea"

diagnosis_prompt = MedicalRAGTemplates.diagnosis_template(medical_context, symptoms)
print("\nDiagnosis Template Example:")
print(diagnosis_prompt)
print(f"Tokens: {count_tokens(diagnosis_prompt)}")

# Medication info template example
medication = "Aspirin"
medication_prompt = MedicalRAGTemplates.medication_info_template(medical_context, medication)
print("\nMedication Info Template Example:")
print(medication_prompt)
print(f"Tokens: {count_tokens(medication_prompt)}")

print("\nCreating domain-specific template libraries allows for:")
print("1. Consistent prompt structure across applications")
print("2. Domain-appropriate language and constraints")
print("3. Built-in safety mechanisms (disclaimers, citation requirements)")
print("4. Easier maintenance and updates to prompting strategy")

print_separator()


**Section 5: Template Versioning and Management**

In [ ]:
print("Section 5: Template Versioning and Management")

class TemplateRegistry:
    """Registry for managing and versioning prompt templates."""

    def __init__(self):
        self.templates = {}
        self.version_history = {}

    def register_template(self, name, template, version="1.0.0"):
        """Register a new template version."""
        if name not in self.templates:
            self.templates[name] = template
            self.version_history[name] = {version: template}
        else:
            self.templates[name] = template
            self.version_history[name][version] = template

    def get_template(self, name, version=None):
        """Get a template by name and optional version."""
        if name not in self.templates:
            raise KeyError(f"Template '{name}' not found")

        if version is None:
            return self.templates[name]

        if version not in self.version_history[name]:
            raise KeyError(f"Version '{version}' not found for template '{name}'")

        return self.version_history[name][version]

    def list_versions(self, name):
        """List all versions of a template."""
        if name not in self.version_history:
            raise KeyError(f"Template '{name}' not found")

        return list(self.version_history[name].keys())

    def get_latest_version(self, name):
        """Get the latest version of a template based on semantic versioning."""
        if name not in self.version_history:
            raise KeyError(f"Template '{name}' not found")

        versions = list(self.version_history[name].keys())
        versions.sort(key=lambda s: [int(u) for u in s.split('.')])
        return versions[-1]

# Create a template registry and add templates
registry = TemplateRegistry()

# Register templates with versioning
qa_template_v1 = """
Answer the question based on the context.

Context: {context}
Question: {question}

Answer:
"""

qa_template_v2 = """
Answer the question based only on the provided context.
If the answer isn't in the context, say "I don't have enough information."

Context: {context}
Question: {question}

Answer:
"""

qa_template_v3 = """
You are an assistant for question-answering tasks.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer concise.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

registry.register_template("qa", qa_template_v1, "1.0.0")
registry.register_template("qa", qa_template_v2, "1.1.0")
registry.register_template("qa", qa_template_v3, "2.0.0")

# Test registry functions
print("Template Registry Example:")

print("\nList of QA template versions:")
versions = registry.list_versions("qa")
print(versions)

print("\nLatest QA template version:")
latest = registry.get_latest_version("qa")
print(latest)

print("\nQA Template v1.0.0:")
v1 = registry.get_template("qa", "1.0.0")
print(v1)

print("\nQA Template v2.0.0:")
v2 = registry.get_template("qa", "2.0.0")
print(v2)

print("\nDefault QA Template (latest):")
default = registry.get_template("qa")
print(default)

# Demonstrate saving and loading the registry
print("\nSaving and loading the registry:")

# Save to JSON
registry_data = {
    "templates": {name: template for name, template in registry.templates.items()},
    "version_history": registry.version_history
}

# Write to disk (uncommenting to save to file)
# with open("template_registry.json", "w") as f:
#     json.dump(registry_data, f, indent=2)

# Load from JSON (for illustration)
loaded_registry = TemplateRegistry()
for name, template in registry_data["templates"].items():
    for version, template_text in registry_data["version_history"][name].items():
        loaded_registry.register_template(name, template_text, version)

print("Registry can be saved and loaded successfully")

print_separator()

**Section 6: Integration with RAG Pipelines**

In [ ]:
print("Section 6: Integration with RAG Pipelines")

# Skip actual integration if LangChain isn't available
print("Integration with RAG Pipelines (conceptual overview):")

print("""
# Typical steps for integrating templates with a RAG pipeline:

1. Define your prompt template:
   prompt = PromptTemplate.from_template(
       "Answer based on context: {context}\\nQ: {question}\\nA:"
   )

2. Create a retriever (typically from a vector store):
   retriever = vector_store.as_retriever()

3. Create an LLM instance:
   llm = ChatOpenAI(temperature=0)

4. Combine into a RAG chain:
   rag_chain = (
       {"context": retriever, "question": RunnablePassthrough()}
       | prompt
       | llm
   )

5. Invoke the chain:
   response = rag_chain.invoke("What is the height of the Eiffel Tower?")
""")

print("\nThis basic pattern can be adapted for different frameworks and complex scenarios.")

print("""
# More advanced integration patterns include:

1. Multi-step reasoning chains:
   - Question reformulation → Retrieval → Answer generation

2. Retrieval customization:
   - Filtering results by metadata
   - Re-ranking documents by relevance
   - Dynamically adjusting retrieval parameters

3. Post-processing:
   - Citation extraction
   - Answer validation
   - Response formatting
""")

print_separator()

**Section 7: Comparing Framework Capabilities**

In [ ]:
print("Section 7: Comparing Framework Capabilities")

print("Comparison of Template Framework Features:")

# Create a comparison table
comparison = {
    "Feature": [
        "Basic Templates",
        "Few-Shot Learning",
        "Chat Templates",
        "XML/JSON Output Parsing",
        "Template Validation",
        "Template Composition",
        "Integration with Retrievers",
        "Custom Output Parsers",
        "Template Reuse",
        "Memory Integration"
    ],
    "LangChain": [
        "✅ Excellent",
        "✅ Built-in",
        "✅ Native Support",
        "✅ Strong Support",
        "✅ Available",
        "✅ Excellent",
        "✅ Seamless",
        "✅ Extensive",
        "✅ Excellent",
        "✅ Built-in"
    ],
    "LlamaIndex": [
        "✅ Excellent",
        "⚠️ Limited",
        "✅ Available",
        "⚠️ Basic",
        "❌ Limited",
        "✅ Good",
        "✅ Native",
        "⚠️ Basic",
        "✅ Good",
        "✅ Available"
    ],
    "Haystack": [
        "✅ Good",
        "❌ Limited",
        "⚠️ Basic",
        "⚠️ Basic",
        "❌ Limited",
        "✅ Pipeline-based",
        "✅ Native",
        "⚠️ Basic",
        "⚠️ Limited",
        "⚠️ Basic"
    ]
}

# Print comparison table
print("\nTemplate Framework Feature Comparison:")
header = f"{'Feature':<25} | {'LangChain':<15} | {'LlamaIndex':<15} | {'Haystack':<15}"
separator = "-" * len(header)
print(separator)
print(header)
print(separator)

for i in range(len(comparison["Feature"])):
    row = f"{comparison['Feature'][i]:<25} | {comparison['LangChain'][i]:<15} | {comparison['LlamaIndex'][i]:<15} | {comparison['Haystack'][i]:<15}"
    print(row)

print(separator)

print("\nFramework Selection Guidelines:")
print("1. LangChain: Best for complex applications requiring flexible prompt engineering and chain composition")
print("2. LlamaIndex: Excellent for document-centric applications with complex retrieval needs")
print("3. Haystack: Good for production deployments requiring stable, modular pipelines")

print("\nConsiderations for Framework Selection:")
print("- Project complexity and scale")
print("- Integration with existing systems")
print("- Team familiarity with the framework")
print("- Specific RAG features needed (retrieval methods, prompt flexibility, etc.)")
print("- Production deployment requirements")

print_separator()

**Section 8: Putting It All Together**

In [ ]:
print("Section 8: Putting It All Together")

class RAGPromptManager:
    """
    Comprehensive prompt manager for RAG applications.
    Combines template registry, versioning, and framework integration.
    """

    def __init__(self):
        self.registry = TemplateRegistry()
        self._register_default_templates()

    def _register_default_templates(self):
        """Register default templates for common RAG tasks."""
        # QA template
        qa_template = """
        You are an assistant for question-answering tasks.
        Use the following pieces of context to answer the question at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.

        CONTEXT:
        {context}

        QUESTION:
        {question}

        ANSWER:
        """

        # Summarization template
        summarize_template = """
        Create a comprehensive summary of the following information.

        CONTEXT:
        {context}

        Provide a concise summary that captures the key points.
        Focus on the main ideas and significant details.

        SUMMARY:
        """

        # Comparison template
        compare_template = """
        Compare and contrast the following entities based on the provided information:

        CONTEXT:
        {context}

        ENTITIES TO COMPARE:
        {entity_1}
        {entity_2}

        Provide a structured comparison highlighting key similarities and differences.

        COMPARISON:
        """

        # Register templates
        self.registry.register_template("qa", qa_template, "1.0.0")
        self.registry.register_template("summarize", summarize_template, "1.0.0")
        self.registry.register_template("compare", compare_template, "1.0.0")

    def register_custom_template(self, name, template, version="1.0.0"):
        """Register a custom template."""
        self.registry.register_template(name, template, version)

    def get_template(self, name, version=None):
        """Get a template by name and optional version."""
        return self.registry.get_template(name, version)

    def create_prompt(self, template_name, version=None, **kwargs):
        """Create a formatted prompt from a template with provided variables."""
        template = self.get_template(template_name, version)
        formatted = template.format(**kwargs)

        # Print token usage stats
        token_count = count_tokens(formatted)
        print(f"Prompt '{template_name}' generated with {token_count} tokens")

        return formatted

    def integrate_with_langchain(self, template_name, version=None):
        """Create a LangChain PromptTemplate from the registry."""
        try:
            from langchain.prompts import PromptTemplate as LangChainPrompt

            template_str = self.get_template(template_name, version)

            # Parse input variables
            import re
            input_vars = re.findall(r'\{([^{}]*)\}', template_str)

            return LangChainPrompt(
                template=template_str,
                input_variables=input_vars
            )
        except ImportError:
            print("LangChain not available")
            return None

# Create and demonstrate the RAG Prompt Manager
prompt_manager = RAGPromptManager()

print("RAG Prompt Manager Demo:")

# Get a template
qa_template = prompt_manager.get_template("qa")
print("\nQA Template:")
print(qa_template)

# Create a formatted prompt
context = "\n".join([doc.page_content for doc in sample_docs])
formatted_qa = prompt_manager.create_prompt("qa", context=context, question="What is the height of the Eiffel Tower?")
print("\nFormatted QA prompt:")
print(formatted_qa)

# Register a custom template
custom_template = """
You are a specialized assistant for {domain} questions.
Use the following information to provide a detailed response.

CONTEXT:
{context}

QUESTION:
{question}

Provide a comprehensive answer with detailed {domain} analysis.
Include any relevant {domain} terminology and concepts.

DETAILED RESPONSE:
"""

prompt_manager.register_custom_template("domain_specific", custom_template)

formatted_custom = prompt_manager.create_prompt(
    "domain_specific",
    domain="architectural",
    context=context,
    question="Describe the Eiffel Tower's design."
)

print("\nCustom domain-specific prompt:")
print(formatted_custom)

# Demonstrate framework integration (conceptual)
print("\nIntegrating with LangChain (conceptual overview):")
print("""
To integrate the templates with a complete RAG pipeline:

1. Convert the template to a LangChain PromptTemplate:
   lc_prompt = prompt_manager.integrate_with_langchain("qa")

2. Create a retriever from a vector store:
   retriever = vector_store.as_retriever()

3. Create an LLM:
   llm = ChatOpenAI()

4. Build the RAG chain:
   from langchain.chains import RetrievalQA

   rag_chain = RetrievalQA.from_chain_type(
       llm=llm,
       chain_type="stuff",
       retriever=retriever,
       chain_type_kwargs={"prompt": lc_prompt}
   )

5. Run the chain:
   rag_chain.invoke({"query": "What is the Eiffel Tower?"})
""")

print_separator()

print("Notebook completed!")